In [1]:
# importing packages and libaray
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import importlib
import Ganmodel


2.1.0


In [2]:
# defining path for accessing dataset
image_dir = 'H:/Downloads/keras_png_slices_data/Dataset/keras_png_slices_seg_train/image'


In [3]:
# function for loading images.
def data_gen(img_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    np.random.shuffle(n)
    print("Number of Train_images:",len(n))
    
    while (True):
        img = np.zeros((batch_size, 256, 256, 1)).astype('float')
       
        for i in range(c, c+batch_size): #Reading set of images for a mentioned batch-size . 
            train_img = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(
                img_folder+'/'+n[i],color_mode='grayscale', target_size=(256, 256)))/255
            
            img[i-c] = train_img #add to array - img[0], img[1], and so on.
             
        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            np.random.shuffle(n)
                  
        return img

In [4]:
# Creating Data-generator to load images
train_gen = data_gen(image_dir, batch_size = 16)
print("Shape of Train image: ",train_gen[1].shape)

Number of Train_images: 9664
Shape of Train image:  (256, 256, 1)


In [5]:
#Build Generator and Discriminator
generator = Ganmodel.make_generator_model()
discriminator = Ganmodel.make_discriminator_model()

In [6]:
generator.summary(),discriminator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 1)       51720897  
Total params: 51,720,897
Trainable params: 51,588,833
Non-trainable params: 132,064
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 1580929   
Total params: 1,580,929
Trainable params: 1,580,929
Non-trainable params: 0
_________________________________________________________________


(None, None)